In [1]:
import bartpy.tools as bart
from matplotlib import pyplot as plt

%matplotlib inline

In [2]:
traj_rad = bart.traj(r=True, x=512, y=64)

/Users/malits/bart_work/bart/bart traj -x 512 -y 64 -r   output  


In [3]:
traj_rad2 = bart.scale(traj_rad, 0.5)

/Users/malits/bart_work/bart/bart scale   0.5 input output  


In [4]:
ksp_sim = bart.phantom(k=True, s=8, t=traj_rad2)

/Users/malits/bart_work/bart/bart phantom -s 8 -k -t t  output  


In [5]:
trad_rad2 = bart.scale(traj_rad, 0.6)

/Users/malits/bart_work/bart/bart scale   0.6 input output  


In [6]:
igrid = bart.nufft(traj_rad2, ksp_sim, i=True, t=True)

/Users/malits/bart_work/bart/bart nufft -i -t   traj input output  


In [7]:
reco1 = bart.rss(igrid, 8)

/Users/malits/bart_work/bart/bart rss   8 input output  


In [8]:
lowres_img = bart.nufft(traj_rad2, ksp_sim, i=True, d=[24,24,1], t=True)
lowres_ksp = bart.fft(lowres_img, 7, u=True)

/Users/malits/bart_work/bart/bart nufft -i -d 24:24:1 -t   traj input output  
/Users/malits/bart_work/bart/bart fft -u   7 input output  


In [9]:
ksp_zerop = bart.resize(lowres_ksp, [0, 1], [308, 308], c=True)

/Users/malits/bart_work/bart/bart resize -c  0 308 1 308 input output  


In [10]:
sens = bart.ecalib(ksp_zerop, m=1)

/Users/malits/bart_work/bart/bart ecalib -m 1   kspace sensitivities  


In [11]:
reco2 = bart.pics(ksp_sim, sens, S=True, r=0.001, t=traj_rad2)

/Users/malits/bart_work/bart/bart pics -r 0.001 -t t-S   kspace sensitivities output  


(308, 308, 1, 8)